In [1]:

from qwen3_moe_fused.fast_lora import patch_Qwen3MoeFusedSparseMoeBlock_forward
from qwen3_moe_fused.lora import patch_lora_config
from qwen3_moe_fused.quantize.quantizer import patch_bnb_quantizer
from qwen3_moe_fused.modular_qwen3_moe_fused import Qwen3MoeFusedForCausalLM
import os

from transformers import BitsAndBytesConfig, AutoTokenizer
 
import torch
from peft import PeftModel 
# # === Step 1. 打补丁 ===

patch_bnb_quantizer()
# patch_Qwen3MoeFusedSparseMoeBlock_forward()
os.environ.setdefault("TRITON_PRINT_AUTOTUNING", "0")


This can be used to load a bitsandbytes version built with a CUDA version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=

/media/gpt4-pdf-chatbot-langchain/pyenv-qwen3-moe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'0'

In [3]:
 

model_id = "/media/checkpoint/Qwen3-30B-A3B-Instruct-2507"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="fp4",
    bnb_4bit_use_double_quant=True,
)


dtype = torch.float16
model = Qwen3MoeFusedForCausalLM.from_pretrained(model_id,  torch_dtype=dtype, quantization_config=bnb_config,
    offload_folder="./offload",
    offload_state_dict=True,)


Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████| 16/16 [01:43<00:00,  6.49s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 384.00 MiB. GPU 0 has a total capacity of 23.56 GiB of which 202.50 MiB is free. Including non-PyTorch memory, this process has 22.72 GiB memory in use. Of the allocated memory 21.85 GiB is allocated by PyTorch, and 588.16 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [3]:

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [12]:

from peft import PeftConfig, PeftModel


patch_lora_config()
lora_dir = "/media/gpt4-pdf-chatbot-langchain/transformers-qwen3-moe-fused/moe_unsloth/checkpoint-300"

# 载入 LoRA 配置
peft_config = PeftConfig.from_pretrained(lora_dir)

# 指定要排除的模块，例如 embedding 和 lm_head
peft_config.target_modules = [
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj"
]  # 手动控制，只保留注意力和 MLP 层
 
model = PeftModel.from_pretrained(model, lora_dir,
    config=peft_config
)

/media/gpt4-pdf-chatbot-langchain/pyenv-qwen3-moe/lib/python3.10/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


TypeError: LoraMoeFusedLinear.update_layer() got an unexpected keyword argument 'use_dora'

In [4]:

messages = [
    
            {
                "role": "system",
                "content": "\nYou are designed to help with a variety of tasks, from answering questions     to providing summaries to other types of analyses.\n\n## Tools\nYou have access to a wide variety of tools. You are responsible for using\nthe tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools\nto complete each subtask.\n\nYou have access to the following tools:\n> Tool Name: resume_zhangmaofeng\nTool Description: 关于张毛峰的简历信息，包括了langchain-chatchat、InterpretationoDreams、KM 平台、省检修特高压生产指挥管控系统、智能运检移动应用、福建监控系统项目eg:检索时带上详细的问题内容\nTool Args: {\"type\": \"object\", \"properties\": {\"input\": {\"title\": \"Input\", \"type\": \"string\"}}, \"required\": [\"input\"]}\n\n> Tool Name: resume_liulijian\nTool Description: 关于刘立兼的简历信息，包括了•篝火心理、雷鸟365、雷鸟365、网聚宝CRM、AP数据基盘、AP数据基盘等项目eg:检索时带上详细的问题内容\nTool Args: {\"type\": \"object\", \"properties\": {\"input\": {\"title\": \"Input\", \"type\": \"string\"}}, \"required\": [\"input\"]}\n\n> Tool Name: resume_songjinke\nTool Description: 关于宋金珂的简历信息，包括了全球 IPv4 空间内的物联网设备扫描识别和隐私安全分、开源软件生态内的跨项目依赖分析及漏洞影响追溯、已发表论文列表、IoT 设备安全等项目eg:检索时带上详细的问题内容\nTool Args: {\"type\": \"object\", \"properties\": {\"input\": {\"title\": \"Input\", \"type\": \"string\"}}, \"required\": [\"input\"]}\n\n\n## Output Format\nTo answer the question, please use the following format.\n\n```\nThought: I need to use a tool to help me answer the question.\nAction: tool name (one of resume_zhangmaofeng, resume_liulijian, resume_songjinke) if using a tool.\nAction Input: the input to the tool, in a JSON format representing the kwargs (e.g. {\"input\": \"hello world\", \"num_beams\": 5})\n```\n\nPlease ALWAYS start with a Thought.\n\nPlease use a valid JSON format for the Action Input. Do NOT do this {'input': 'hello world', 'num_beams': 5}.\n\nIf this format is used, the user will respond in the following format:\n\n```\nObservation: tool response\n```\n\nYou should keep repeating the above format until you have enough information\nto answer the question without using any more tools. At that point, you MUST respond\nin the one of the following two formats:\n\n```\nThought: I can answer without using any more tools.\nAnswer: [your answer here]\n```\n\n```\nThought: I cannot answer the question with the provided tools.\nAnswer: Sorry, I cannot answer your query.\n```\n\n## Current Conversation\nBelow is the current conversation consisting of interleaving human and assistant messages.\n\n"
            },
            {
                "role": "user",
                "content": "请问候选人的全名是什么？"
            },
            {
                "role": "assistant",
                "content": "Thought: 我需要使用工具来获取候选人的全名。\nAction: resume_zhangmaofeng\nAction Input: {'input': '请问候选人的全名是什么？'}"
            },
            {
                "role": "user",
                "content": "Observation: The stroke count of Zhang Maofeng's name is 24 strokes for \"张毛峰\". Zhang (张) has 11 strokes, Mao (毛) has 4 strokes, and Feng (峰) has 9 strokes."
            }
        
 
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    input_ids = inputs, streamer = text_streamer, max_new_tokens = 128
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Thought and and and and
3::

l::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::


In [9]:

# prepare the model input
prompt = "你好."
messages = [
    
    {
        "role": "system",
        "content": "\nYou are designed to help with a variety of tasks, from answering questions     to providing summaries to other types of analyses.\n\n## Tools\nYou have access to a wide variety of tools. You are responsible for using\nthe tools in any sequence you deem appropriate to complete the task at hand.\nThis may require breaking the task into subtasks and using different tools\nto complete each subtask.\n\nYou have access to the following tools:\n> Tool Name: resume_zhangmaofeng\nTool Description: 关于张毛峰的简历信息，包括了langchain-chatchat、InterpretationoDreams、KM 平台、省检修特高压生产指挥管控系统、智能运检移动应用、福建监控系统项目eg:检索时带上详细的问题内容\nTool Args: {\"type\": \"object\", \"properties\": {\"input\": {\"title\": \"Input\", \"type\": \"string\"}}, \"required\": [\"input\"]}\n\n> Tool Name: resume_liulijian\nTool Description: 关于刘立兼的简历信息，包括了•篝火心理、雷鸟365、雷鸟365、网聚宝CRM、AP数据基盘、AP数据基盘等项目eg:检索时带上详细的问题内容\nTool Args: {\"type\": \"object\", \"properties\": {\"input\": {\"title\": \"Input\", \"type\": \"string\"}}, \"required\": [\"input\"]}\n\n> Tool Name: resume_songjinke\nTool Description: 关于宋金珂的简历信息，包括了全球 IPv4 空间内的物联网设备扫描识别和隐私安全分、开源软件生态内的跨项目依赖分析及漏洞影响追溯、已发表论文列表、IoT 设备安全等项目eg:检索时带上详细的问题内容\nTool Args: {\"type\": \"object\", \"properties\": {\"input\": {\"title\": \"Input\", \"type\": \"string\"}}, \"required\": [\"input\"]}\n\n\n## Output Format\nTo answer the question, please use the following format.\n\n```\nThought: I need to use a tool to help me answer the question.\nAction: tool name (one of resume_zhangmaofeng, resume_liulijian, resume_songjinke) if using a tool.\nAction Input: the input to the tool, in a JSON format representing the kwargs (e.g. {\"input\": \"hello world\", \"num_beams\": 5})\n```\n\nPlease ALWAYS start with a Thought.\n\nPlease use a valid JSON format for the Action Input. Do NOT do this {'input': 'hello world', 'num_beams': 5}.\n\nIf this format is used, the user will respond in the following format:\n\n```\nObservation: tool response\n```\n\nYou should keep repeating the above format until you have enough information\nto answer the question without using any more tools. At that point, you MUST respond\nin the one of the following two formats:\n\n```\nThought: I can answer without using any more tools.\nAnswer: [your answer here]\n```\n\n```\nThought: I cannot answer the question with the provided tools.\nAnswer: Sorry, I cannot answer your query.\n```\n\n## Current Conversation\nBelow is the current conversation consisting of interleaving human and assistant messages.\n\n"
    },
    {
        "role": "user",
        "content": "候选人的联系电话是多少？"
    },
    {
        "role": "assistant",
        "content": "Thought: 我需要使用工具来查找候选人的联系电话。由于这个问题没有指定具体的候选人，我需要决定使用哪个简历工具。这里我假设是在询问张毛峰的联系电话。\nAction: resume_zhangmaofeng\nAction Input: {'input': '请提供张毛峰的联系电话。'}"
    },
    {
        "role": "user",
        "content": "Observation: （+86）17621312993"
    }
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
    use_cache = True 
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Thought and                                                                                                                              


In [15]:

# prepare the model input
prompt = "你可以搞"
messages = [
    
    {
        "role": "user",
        "content": prompt
    }
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(
    input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
    temperature=0.8,
        top_p=0.9
)

你好 je , 

d
k195.



l19.3d =2 = de
ll sum = :
22 = a, : = 

KeyboardInterrupt: 